In [ ]:
import os
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import math
import time
import tensorflow as tf
from tensorflow.keras.layers import GRU, LSTM, Bidirectional, Dense, Flatten, Conv1D, BatchNormalization, LeakyReLU, Dropout
from tensorflow.keras import Sequential
#from keras.utils.vis_utils import plot_model
from pickle import load
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import statsmodels.api as sm
from math import sqrt
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from pickle import dump
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import unicodedata

import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load text data

In [5]:
rpath = "/content/drive/MyDrive/UofT/grad/ece1508/Final_Project/sentiment_analysis/"

In [25]:
all_tweets = pd.read_csv(rpath + 'stock_tweets.csv')

In [7]:
print(all_tweets.shape)
all_tweets.head()

(80793, 4)


,Date,Tweet,Stock Name,Company Name
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc."
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc."
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc."
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc."
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc."


In [9]:
stock_name = 'AAPL'
df = all_tweets[all_tweets['Stock Name'] == stock_name]
print(df.shape)
df.head()

(5056, 4)


,Date,Tweet,Stock Name,Company Name
55958,2022-09-29 22:23:54+00:00,$NIO just because I'm down money doesn't mean ...,AAPL,Apple Inc.
55959,2022-09-29 20:37:01+00:00,After trading for 9+ years \n\nThis is how to ...,AAPL,Apple Inc.
55960,2022-09-29 20:19:43+00:00,Not something you see very often. \n-2 ATR mo...,AAPL,Apple Inc.
55961,2022-09-29 20:13:48+00:00,$AAPL was down almost 5% today. Rare to see it...,AAPL,Apple Inc.
55962,2022-09-29 19:50:00+00:00,$AAPL APPLE JUST FIRED ITS HEAD OF PROCUREMENT...,AAPL,Apple Inc.


In [16]:
sent_df = df.copy()
sent_df["sentiment_score"] = ''
sent_df["Negative"] = ''
sent_df["Neutral"] = ''
sent_df["Positive"] = ''
sent_df.head()

,Date,Tweet,Stock Name,Company Name,sentiment_score,Negative,Neutral,Positive
55958,2022-09-29 22:23:54+00:00,$NIO just because I'm down money doesn't mean ...,AAPL,Apple Inc.,,,,
55959,2022-09-29 20:37:01+00:00,After trading for 9+ years \n\nThis is how to ...,AAPL,Apple Inc.,,,,
55960,2022-09-29 20:19:43+00:00,Not something you see very often. \n-2 ATR mo...,AAPL,Apple Inc.,,,,
55961,2022-09-29 20:13:48+00:00,$AAPL was down almost 5% today. Rare to see it...,AAPL,Apple Inc.,,,,
55962,2022-09-29 19:50:00+00:00,$AAPL APPLE JUST FIRED ITS HEAD OF PROCUREMENT...,AAPL,Apple Inc.,,,,


## Get sentiment score

In [17]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [18]:
%%time
sentiment_analyzer = SentimentIntensityAnalyzer()
for indx, row in sent_df.T.iteritems():
    try:
        sentence_i = unicodedata.normalize('NFKD', sent_df.loc[indx, 'Tweet'])
        sentence_sentiment = sentiment_analyzer.polarity_scores(sentence_i)
        sent_df.at[indx, 'sentiment_score'] = sentence_sentiment['compound']
        sent_df.at[indx, 'Negative'] = sentence_sentiment['neg']
        sent_df.at[indx, 'Neutral'] = sentence_sentiment['neu']
        sent_df.at[indx, 'Positive'] = sentence_sentiment['pos']
    except TypeError:
        print (sent_df.loc[indx, 'Tweet'])
        print (indx)
        break

CPU times: user 2.62 s, sys: 1 ms, total: 2.62 s
Wall time: 2.68 s


In [19]:
sent_df

,Date,Tweet,Stock Name,Company Name,sentiment_score,Negative,Neutral,Positive
55958,2022-09-29 22:23:54+00:00,$NIO just because I'm down money doesn't mean ...,AAPL,Apple Inc.,0.25,0.158,0.684,0.158
55959,2022-09-29 20:37:01+00:00,After trading for 9+ years \n\nThis is how to ...,AAPL,Apple Inc.,0.3612,0.0,0.915,0.085
55960,2022-09-29 20:19:43+00:00,Not something you see very often. \n-2 ATR mo...,AAPL,Apple Inc.,0.0,0.0,1.0,0.0
55961,2022-09-29 20:13:48+00:00,$AAPL was down almost 5% today. Rare to see it...,AAPL,Apple Inc.,-0.25,0.182,0.699,0.119
55962,2022-09-29 19:50:00+00:00,$AAPL APPLE JUST FIRED ITS HEAD OF PROCUREMENT...,AAPL,Apple Inc.,-0.6523,0.143,0.857,0.0
...,...,...,...,...,...,...,...,...
61009,2021-09-30 15:37:52+00:00,"All about this trendline now on $AAPL, continu...",AAPL,Apple Inc.,-0.4019,0.197,0.803,0.0
61010,2021-09-30 15:30:03+00:00,This thread is just a broad overview of the ba...,AAPL,Apple Inc.,0.4767,0.0,0.892,0.108
61011,2021-09-30 13:36:10+00:00,"In 2020, Tim Cook, CEO of $AAPL, earned $265,0...",AAPL,Apple Inc.,0.0,0.0,1.0,0.0
61012,2021-09-30 10:22:53+00:00,The media is really pushing hard for their big...,AAPL,Apple Inc.,0.2296,0.076,0.829,0.095


In [20]:
sent_df['Date'] = pd.to_datetime(sent_df['Date'])
sent_df['Date'] = sent_df['Date'].dt.date
sent_df = sent_df.drop(columns=['Negative', 'Positive', 'Neutral', 'Stock Name', 'Company Name'])
sent_df.head()

,Date,Tweet,sentiment_score
55958,2022-09-29,$NIO just because I'm down money doesn't mean ...,0.25
55959,2022-09-29,After trading for 9+ years \n\nThis is how to ...,0.3612
55960,2022-09-29,Not something you see very often. \n-2 ATR mo...,0.0
55961,2022-09-29,$AAPL was down almost 5% today. Rare to see it...,-0.25
55962,2022-09-29,$AAPL APPLE JUST FIRED ITS HEAD OF PROCUREMENT...,-0.6523


In [21]:
score_df = sent_df.groupby([sent_df['Date']]).mean()
print(score_df.shape)

(365, 1)


In [22]:
score_df

,sentiment_score
Date,
2021-09-30,0.098900
2021-10-01,0.291891
2021-10-02,0.007525
2021-10-03,0.834400
2021-10-04,0.047410
...,...
2022-09-25,0.142021
2022-09-26,0.289800
2022-09-27,0.256164


## Processing Input Data

In [ ]:
def preprocessing(data, stock, days, features=[0,1,2,3]):
    #Get stock data for specific stock
    stock_data = data[data['ticker_symbol'] == stock].copy()

    #Drops features not in feature list
    dropped_features = []
    for i in range(4):
        if i not in features:
            dropped_features.append(i+3)
    stock_data.drop(columns = stock_data.columns[dropped_features], axis=1, inplace=True)

    #Split data into testing and training, using 8 years and 2 years from Train/Test Split
    training_data_0 = stock_data[stock_data['day_date'] < '2019-01-01'].copy()
    testing_data = stock_data[stock_data['day_date'] >= '2019-01-01'].copy()

    training_data = training_data_0.drop(['ticker_symbol', 'day_date'], axis=1)
    scaler = MinMaxScaler()
    training_data = scaler.fit_transform(training_data)
    scale = 1/scaler.scale_[0]

    X_train = []
    y_train = []

    for i in range(days, training_data.shape[0]):
        X_train.append(training_data[i-days:i])
        y_train.append(training_data[i, 0])
    X_train, y_train = np.array(X_train), np.array(y_train)

    past_days = training_data_0.tail(days)
    inputs = past_days.append(testing_data, ignore_index=True)
    inputs = inputs.drop(['ticker_symbol', 'day_date'], axis=1)
    inputs = scaler.transform(inputs)

    X_test = []
    y_test = []

    for i in range(days, inputs.shape[0]):
        X_test.append(inputs[i-days:i])
        y_test.append(inputs[i, 0])
    X_test, y_test = np.array(X_test), np.array(y_test)

    return X_train, y_train, X_test, y_test, scale

## LSTM

In [23]:
def predict_stocks_LSTM(X_train, y_train, X_test, y_test, scale, days, hidden_layers, nodes, dropout, training_epochs):

    model = Sequential()

    model.add(LSTM(units = days, activation = 'relu', return_sequences = True, input_shape = (X_train.shape[1], X_train[0].shape[1])))
    model.add(Dropout(dropout))

    for i in range(hidden_layers):
        if (i == hidden_layers - 1):
            model.add(LSTM(units = nodes[i], activation = 'relu'))
        else:
            model.add(LSTM(units = nodes[i], activation = 'relu', return_sequences = True))
        model.add(Dropout(dropout))

    model.add(Dense(units = 1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=training_epochs, batch_size=32)

    y_pred = model.predict(X_test)

    y_pred = y_pred*scale
    y_test = y_test*scale

    plt.figure(figsize=(14,5))
    plt.plot(y_test, color='red', label='Real Stock Price')
    plt.plot(y_pred, color='green', label='Predicted Stock Price')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.show()

    rmse = np.sqrt((1/(y_pred.size)) * np.sum(np.square(y_pred - y_test)))
    print(rmse)
    return rmse